## Sprawozdanie5

**Data:** 2024-06-04
**Autor:** Piotr Szepietowski

A = [[4,2,-1],[2,5,2],[-1,2,10]] 
b = [12, -8, 4]

In [ ]:
import numpy as np
function [x] = conjgrad(A,b,x0)
r = b - A*x0;
w = -r;
z = A*w;
a = (r'*w)/(w'*z);
x = x0 + a*w;

for i = 1:size(A,1);
    r = r - a*z;
    if( norm(r) < 1e-10 )
        break;
    end
    B = (r'*z)/(w'*z);
    w = -r + B*w;
    z = A*w;
    a = (r'*w)/(w'*z);
    x = x + a*w;
end

end

def gradient(A, b):
    x0 = [0 for x in range(len(b))]
    r = b - A*x0
    w = -1 * r
    z = A * w
    a = 

    for i in range(A)